In [35]:
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [36]:
inputs = pd.read_csv("inputs.csv")
inputs["MR"] = ""
weather  = pd.read_csv("MERRA_PRECIP_HOUR.csv")
weather["DATE"] = weather.DATE_TIME.apply(lambda x: x[:9])


#fixed value 
a = -0.3123
b = 0.3
km = 6.8157

In [37]:
inputs

,Latitude,Longitude,Cell No/identifier,date,SOPT,MROPT,hydrlic_conduc,MR
0,45.264827,-93.714984,127,08-FEB-22,0.2515,30000,7.390,
1,45.263060,-93.711164,188,08-FEB-22,0.2625,28000,0.919,
2,45.264593,-93.713997,189,08-FEB-22,0.3448,25000,0.739,
3,45.259115,-93.703928,328,08-FEB-22,0.3000,25000,0.738,


In [38]:
weather

,DATE_TIME,RUNOFF,PRECIPITATION,PRECIP_FLUX,DATE
0,07-FEB-22 12.00.00.000000 PM,0.000000,0.004,1.245600e-06,07-FEB-22
1,07-FEB-22 01.00.00.000000 PM,0.000000,0.001,2.889000e-07,07-FEB-22
2,07-FEB-22 02.00.00.000000 PM,0.000000,0.000,1.214000e-07,07-FEB-22
3,07-FEB-22 03.00.00.000000 PM,0.000000,0.001,3.048000e-07,07-FEB-22
4,07-FEB-22 04.00.00.000000 PM,0.000000,0.002,5.367000e-07,07-FEB-22
...,...,...,...,...,...
376939,07-AUG-22 02.00.00.000000 PM,0.139793,0.105,2.921370e-05,07-AUG-22
376940,07-AUG-22 03.00.00.000000 PM,0.066835,0.250,6.955860e-05,07-AUG-22
376941,07-AUG-22 04.00.00.000000 PM,0.018388,1.087,3.019570e-04,07-AUG-22
376942,07-AUG-22 05.00.00.000000 PM,0.043496,3.405,9.458070e-04,07-AUG-22


In [39]:
def saturation_cal(cell_no, rain_int, rain_dur, hydrlic_conduc=None):
    if cell_no==188:
        saturation = 22.31+2.70*rain_int+3.31*rain_dur
    
    elif cell_no==189:
        saturation = 20.65+3.61*rain_int+3.15*rain_dur
    
    elif cell_no==127:
        saturation = 18.97+3.27*rain_int+2.71*rain_dur
    
    else:
        if isinstance(hydrlic_conduc, (int, float)): 
            saturation = 21.63+3.60*rain_int+2.83*rain_dur-0.34*hydrlic_conduc
        else:
            raise Exception("Provide correct hydraulic conductivity value")
            
    return saturation

In [40]:
for index, row in inputs.iterrows():
    cell_no = row["Cell No/identifier"] 
    date = row["date"].upper()
    SOPT = row["SOPT"]
    MROPT = row["MROPT"]
    hydrlic_conduc = row["hydrlic_conduc"]
    
    
    #weather information extracts here
    day_weather = weather.loc[weather["DATE"] == date]
    rain_dur = (day_weather['PRECIPITATION'] != 0).sum()
    rain_int = day_weather['PRECIPITATION'].sum()/rain_dur
    
    #saturation and MR calculation
    
    saturation = saturation_cal(cell_no, rain_int, rain_dur, hydrlic_conduc=hydrlic_conduc)    
    right_side= a+(b-a)/(1+math.exp(math.log(-b/a)+km*(saturation-SOPT)))
    MR = MROPT*math.pow(10, right_side)
       
    inputs["MR"][index] = MR

In [41]:
inputs

,Latitude,Longitude,Cell No/identifier,date,SOPT,MROPT,hydrlic_conduc,MR
0,45.264827,-93.714984,127,08-FEB-22,0.2515,30000,7.390,14615.755009
1,45.263060,-93.711164,188,08-FEB-22,0.2625,28000,0.919,13641.371342
2,45.264593,-93.713997,189,08-FEB-22,0.3448,25000,0.739,12179.795841
3,45.259115,-93.703928,328,08-FEB-22,0.3000,25000,0.738,12179.795841


In [42]:
rain_dur, rain_int

(17, 0.01188235294117647)